In [4]:
!ls -al

total 488
drwxr-xr-x 1 ASUS 197121     0 Jul 30 17:36 .
drwxr-xr-x 1 ASUS 197121     0 Jul 30 12:01 ..
drwxr-xr-x 1 ASUS 197121     0 Jul 30 17:14 .ipynb_checkpoints
-rw-r--r-- 1 ASUS 197121  1285 Jul 30 17:36 hand-gesture-recognition.ipynb
drwxr-xr-x 1 ASUS 197121     0 Jul 30 13:33 train
-rw-r--r-- 1 ASUS 197121 39651 Jul 30 12:01 train.csv
drwxr-xr-x 1 ASUS 197121     0 Jul 30 13:35 val
-rw-r--r-- 1 ASUS 197121  5970 Jul 30 12:01 val.csv


In [ ]:
class modelBuilder(metaclass = abc.ABCMeta):
    #initilaizing project data path
    def initialize_path(self, project_folder):
        self.train_doc = np.random.permutation(open(project_folder + '/' + 'train.csv').readlines())
        self.val_doc = np.randoml.permutation(open(project_folder + '/' + 'val.csv').readlines())
        self.train_path = project_folder + '/' + 'train'
        self.val_path = projcet_folder + '/' + 'val'
        self.num_train_sequences = len(self.train_doc)
        self.num_val_sequences = len(self.val_doc)
    #initializing image properties
    def initialize_image_properties(self, image_height = 100, image_width = 100):
        self.image_height = image_height
        self.image_width = image_width
        self.channels = 3
        self.num_classes = 5
        self.total_frames = 30
    #inidializing the batch size, sample frames and number of epoch
    def initialize_hyperparameter(self, frame_to_sample=30, batch_size=20, num_epoch=20):
        self.frames_to_sample = frames_to_sample
        self.batch_size = batch_size
        self.num_epoch = num_epoch
        
    #GENERATOR FUNCTION
    def generator(self, source_path, folder_list, augment=False):
        img_idx = np.round(np.linespace(0, self.total_frames-1, self.frames_to_sample)).astype(int)
        batch_size = self.batch_size
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(t)//batch_size            #check
            
            for batch in range(num_batches):
                batch_data, batch_labels = self.one_batch_data(source_path, t, batch, batch_size, img_idx, augment)
                yield batch_data, batch_labels
                
            remaining_seq = len(%)%batch_size          #check
            
            if (remaining_seq) != 0):
                batch_data, batch_labels = self.one_batch_data(source_path, t, num_batches, batch_size, img_idx, augment, remaining_seq)
                yield batch_data, batch_labels
                
    def one_batch_data(self, source_path, t, batch, batch_size, img_idx, augment, remaining_seq = 0):
        seq_len = reamining_seq if reamining_seq else batch_size
        
        barch_data = np.zero((seq_len, len(img_idx), self.image_height, self.image_width, self.channels))    #check
        batch_labels = np.zeros((seq_len, self.num_classes))    #check
        
        if (augment): 
            batch_data_aug = np.zeros((seq_len, len(img_idx), self.image_height, self.image_width, self.channels))   #check
        
        for folder in range(seq_len):
            imgs = os.listdir(source_path + '/' + t[folder + (batch*batch_size)].split(';')[0])       #check
            for idx,item in enumerate(img_idx):
                #image reading and resizing
                image = imread(source_path + '/' + t[folder + (batch*batch_size)].strip().split(';')[0] + '/' + imgs[item]).astype(np.float32)
                image_resized = imresize(image, (self.image_height, self.image_width,3))
                
                #normalizing the images
                batch_data[folder, idx, :, :, 0] = (image_resized[:,:,0])/255
                batch_data[folder, idx, :, :, 1] = (image_resized[:,:,1])/255
                batch_data[folder, idx, :, :, 2] = (image_resized[:,:,2])/255
                
                if (augment) :
                    shifted = cv2.warpAffine(image, 
                                             np.float32([[1, 0, np.random.randint(-30,30)],[0, 1, np.random.randint(-30, 30)]]), 
                                             (image.shape[1], image.shape[0]))
                    
                    gray = cv2.cvtColor(shifted, cv2.COLOR_BGR2GRAY)
                    
                    x0, y0 = np.argwhere(gray > 0).min(axis = 0)
                    x1, y1 = np.argwhere(gray > 0).max(axis = 0)
                    
                    #cropping the images to have the targeted gestures and remove the noise from the images
                    cropped = shifted[x0:x1,y0:y1,:]
                    
                    image_resized = imresize(cropped, (self.image_height, self.image_width, 3))
                    
                    batch_data_aug[folder, idx, :, :, 0] = (image_resized[:,:,0])/255
                    batch_data_aug[folder, idx, :, :, 1] = (image_resized[:,:,1])/255
                    batch_data_aug[folder, idx, :, :, 2] = (image_resized[:,:,2])/255
                    
            batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            
        if (augment) :
            batch_data = np.concatenate([batch_data, batch_data_aug])
            batch_labels = np.concatenate([batch_labels, batch_labels])
            
        return (batch_data, batch_labels)
    
    def train_model(self, model, augment_data = False):
        train_generator = self.generator(self.train_path, self.train_doc, augment = augment_data)
        val_generator = self.generator(self.val_path, self.val_doc)
        
        model_name = 'model_init' + '_' + str(datetime.datetime.now()).replace(' ', '').replace(':', '_') + '/'
        
        if not os.path.exists(model_name):
            os.mkdir(model_name)
            
        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'
        
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose = 1, save_best_only = True, save_weights_only = False, mode = 'auto', period = 1)
        LR = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.2, verbose = 1, patience = 4)
        
        earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1)
        callbacks_list = [checkpoint, LR, earlystop]
        
        if (self.num_train_sequences%self.batch_size) == 0:
            steps_per_epoch = int(self.num_train_sequence/self.batch_size)
        else:
            steps_per_epoch = (self.num_train_sequences//self.batch_size) + 1
            
        if (self.num_val_sequences%self.batch_size) == 0:
            steps_per_epoch = int(self.num_val_sequence/self.batch_size)
        else:
            steps_per_epoch = (self.num_val_sequences//self.batch_size) + 1
            
        history = model.fit_generator(train_generator, steps_per_epoch = steps_per_epoch, epoch = self.num_epoch, verbose = 1, callbacks = callbacks_list,
                                      validation_data = val_generator, validation_steps = validation_steps, class_weight = None, workers = 1, initial_epoch = 0)
        
        return history
    
    
    @abc.abstractmethod
    def define_model(self):
        pass
                                               